In [ ]:
from paths import get_path_wpp, get_path_collected

import tracemalloc as tmall
tmall.start()

In [ ]:
import utils_collection as ut

In [ ]:
PROJECT_NAME="Climate change"
# PROJECT_NAME="COVID-19"

# PROJECT_NAME='Media_sample'

when running, make sure this date and suffix is consistent with the name of the file of choice. 

inside data/collected/ are the files for pages, which this notebook takes as input

In [ ]:

#---------- for climate change
WPP_DATE='2023-08-16'

#---------- for covid-19
# WPP_DATE='2023-09-18'

#---------- for topicwise all/sample
# WPP_DATE='2024-01-22'

In [ ]:
PROCESS_EN=True
PROCESS_OT=True

In [ ]:
PATH_INPUT=get_path_wpp(PROJECT_NAME)

PATHNAME_INPUT=f'{PATH_INPUT}project_pages_{WPP_DATE}.csv'
PATHNAME_INPUT=f'{PATH_INPUT}project_pages_{WPP_DATE}_sample10.csv'

In [ ]:
PATHNAME_INPUT

In [ ]:
PATH_OUTPUT=get_path_wpp(PROJECT_NAME)+'{}/' #<--- will be filled with language

In [ ]:
PATH_OUTPUT

In [ ]:
RETURN_ALL_REVISIONS=False

In [ ]:
SAMPLE_RUN=False

SUFFIX=''

SAMPLE_PERCENTAGE=1

# SUFFIX+='_'+str(SAMPLE_PERCENTAGE)

In [ ]:
if SAMPLE_RUN:
    PATH_OUTPUT+='sample/'

In [ ]:
ut.SOURCE_STORAGE=True
ut.SOURCE_STORAGE_COUNT_ONE_PER_LOCAL_CALL=True

In [ ]:
ut.RUN_EXPLORE_LANG_TEMPLATES=True
ut.RUN_EXPLORE_LANG_NOSOURCE_REF_TEXT=True

In [ ]:
ut.RUN_TEST_ALL=False

ut.RUN_TEST_NOSOURCE_REF_TEXT=True

In [ ]:
import pandas as pd

In [ ]:
# !pip install urlextract

# Loading Project related pages (all)

In [ ]:
WPP_PAGES=pd.read_csv(PATHNAME_INPUT)
WPP_PAGES

### Selecting pages of article (by namespace 0)

In [ ]:
WPP_PAGES=WPP_PAGES[WPP_PAGES.ns==0]   

# Retrieve already queried pages

In [ ]:
if SAMPLE_RUN:
    print("since this is a sample for testing,") 
    print("the dataframe is replaced silently at every call of this notebook")
    EN_ALREADY_RETRIEVED_PAGES=set()
    OT_ALREADY_RETRIEVED_PAGES=set()
    
else:
    if PROCESS_EN:
        print("looking for english data data in folder:", PATH_OUTPUT.format('english'))
        try:
            EN_ALREADY_RETRIEVED_PAGES=set([row.lang+'$'+row.page \
                                            for _, row in pd.read_csv(f'{PATH_OUTPUT.format("english")}pages_revdata{SUFFIX}.csv').iterrows()])
            print("file found\n")
        except:
            print("file not found\n")
            EN_ALREADY_RETRIEVED_PAGES=set()
            
    if PROCESS_OT:
        print("looking for other languages data data in folder:", PATH_OUTPUT.format('other_languages'))
        try:    
            OT_ALREADY_RETRIEVED_PAGES=set([row.lang+'$'+row.page \
                                            for _, row in pd.read_csv(f'{PATH_OUTPUT.format("other_languages")}pages_revdata{SUFFIX}.csv').iterrows()])    
            print("file found\n")
        except:
            print("file not found\n")
            OT_ALREADY_RETRIEVED_PAGES=set()

In [ ]:
print("english retrieved pages:", len(EN_ALREADY_RETRIEVED_PAGES) )
print("other languages retrieved pages:", len(OT_ALREADY_RETRIEVED_PAGES) )

#### Select pages for the sample

In [ ]:
if SAMPLE_RUN:
    SAMPLE_PAGES=WPP_PAGES.title.values[list( range(0, len(WPP_PAGES), int(100/SAMPLE_PERCENTAGE)) )]
else:
    SAMPLE_PAGES=WPP_PAGES.title.values

# Extract url revisions from project pages + lang links

In [ ]:
print("data will be saved in", PATH_OUTPUT.format("LANGUAGE_SPECIFIER"))
print("suffix:", SUFFIX)

In [ ]:
from time import time

In [ ]:
# TO DO
# add page to already retrieved once has finished
# timer whole process inside iterator
# cleaner checkpoint saving for explore lang informations

In [ ]:
import pickle

# for exploratory purposes
# if PROCESS_EN and PROCESS_OT: _t_prefix='all'
# if not PROCESS_EN and PROCESS_OT: _t_prefix='ot'
# if PROCESS_EN and not PROCESS_OT: _t_prefix='en'

In [ ]:
%%time

EN_ADDED_COUNTER=0
OT_ADDED_COUNTER=0

for n_page, en_page_title in enumerate(SAMPLE_PAGES):
    main_time=time()
    if PROCESS_EN:
        if not f'en${en_page_title}' in EN_ALREADY_RETRIEVED_PAGES:
            en_t_start=time()
            ALL_DATA =ut.get_sources_revisions(en_page_title, 'en', return_all_revisions=RETURN_ALL_REVISIONS,
                                               verbose=True, 
                                               verbose_prefix=f"------------{n_page+1:>4d} of {len(SAMPLE_PAGES)} ")

            mode, header='a', False
            if n_page==0 and not len(EN_ALREADY_RETRIEVED_PAGES):
                mode, header='w', True
            ut.save_rv_data(ALL_DATA, PATH_OUTPUT.format('english'), SUFFIX, mode, header)
            EN_ADDED_COUNTER+=1    
            ut._update_timer('___pipeline_english', en_t_start)
            EN_ALREADY_RETRIEVED_PAGES.update([f'en${en_page_title}'])
            print()

    if PROCESS_OT:
        ot_t_start=time()
        LL_DATA=ut.get_language_links_data(en_page_title)
        for n_ll, ll in enumerate(LL_DATA):
            LANG=ll['lang']
            PAGE=ll['*']
            
            if f'{LANG}${PAGE}' in OT_ALREADY_RETRIEVED_PAGES: continue

            ALL_DATA =ut.get_sources_revisions(PAGE, LANG, return_all_revisions=RETURN_ALL_REVISIONS,
                                               verbose=True, 
                                               verbose_prefix=f"{n_ll+1:>3} of {len(LL_DATA):<3} ({n_page+1:>4d} of {len(SAMPLE_PAGES)})")

            mode, header='a', False
            if n_page==0 and n_ll==0 and not len(OT_ALREADY_RETRIEVED_PAGES):
                mode, header='w', True
            ut.save_rv_data(ALL_DATA, PATH_OUTPUT.format('other_languages'), SUFFIX, mode, header)
            OT_ADDED_COUNTER+=1    
            print('\n next ll...', end='\r')
            OT_ALREADY_RETRIEVED_PAGES.update([f'{LANG}${PAGE}'])

        ut._update_timer('___pipeline_other_languages', ot_t_start)

#         for exploratory purposes
# #     if n_page%10==0:
#     if ut.RUN_EXPLORE_LANG_TEMPLATES:
#         with open(f'../data/explore_lang/{_t_prefix}_lang_template_data{SUFFIX}_new.pickle', 'wb') as handle:
#             pickle.dump(ut.EXPLORE_LANG_TEMPLATES, handle)

#     if ut.RUN_EXPLORE_LANG_NOSOURCE_REF_TEXT:    
#         with open(f'../data/explore_lang/{_t_prefix}_lang_nosource_ref_text{SUFFIX}_new.pickle', 'wb') as handle:
#             pickle.dump(ut.EXPLORE_LANG_NOSOURCE_REF_TEXT, handle)
        
    ut._update_timer('whole_process', main_time)

In [ ]:
ut.N_RESPONSE_EXCEPTIONS

In [ ]:
ut.N_RESPONSE_EXCEPTIONS_LL

In [ ]:
OT_ADDED_COUNTER

In [ ]:
EN_ADDED_COUNTER

In [ ]:
ut.GLOBAL_COUNT_RETAINED

In [ ]:
ut.GLOBAL_COUNT_RETAINED/(ut.GLOBAL_COUNT_SKIPPED+ut.GLOBAL_COUNT_RETAINED)

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
ut.dir_performance(normalize='whole_process') 

### Saving exploration results (if required)

In [ ]:
# import pickle

# if PROCESS_EN and PROCESS_OT: _t_prefix='all'
# if not PROCESS_EN and PROCESS_OT: _t_prefix='ot'
# if PROCESS_EN and not PROCESS_OT: _t_prefix='en'
    
# if ut.RUN_EXPLORE_LANG_TEMPLATES:
#     with open(f'../data/explore_lang/{_t_prefix}_lang_template_data{SUFFIX}_part_3.pickle', 'wb') as handle:
#         pickle.dump(ut.EXPLORE_LANG_TEMPLATES, handle)
        
# if ut.RUN_EXPLORE_LANG_NOSOURCE_REF_TEXT:    
#     with open(f'../data/explore_lang/{_t_prefix}_lang_nosource_ref_text{SUFFIX}_part_3.pickle', 'wb') as handle:
#         pickle.dump(ut.EXPLORE_LANG_NOSOURCE_REF_TEXT, handle)

In [ ]:
# # Load data (deserialize)
# with open(f'../data/other_languages_template_data{SUFFIX}.pickle', 'rb') as handle:
#     this = pickle.load(handle)

#### Extimate parallelization limitations
- Tracing request rate (can use utils timer and counter)
- Tracing memory usage (can use tmall)

In [ ]:
print("request per hour (revisions):", \
      ut.TEMP_COUNTERS['_request_revision_data']/(ut.TEMP_TIMERS['whole_process']/60**2)," requests per hour")
print("request per hour (revisions+language links):", \
      (ut.TEMP_COUNTERS['_request_revision_data']+ut.TEMP_COUNTERS['request_language_links'])/(ut.TEMP_TIMERS['whole_process']/60**2)," requests per hour")

In [ ]:
_, PT_MPK=tmall.get_traced_memory() 

print(f" memory peak: {PT_MPK/(10**6)} Mb")

In [ ]:
_/10**6

In [ ]:
import sys
sys.getsizeof(ut.EXPLORE_LANG_NOSOURCE_REF_TEXT)

# Extimate by sample proportion (rough)

In [ ]:
if SAMPLE_RUN:
    print(f"extimated total time: {ut.TEMP_TIMERS['whole_process']*100/SAMPLE_PERCENTAGE/60**2:.2f} hrs")

# Code performances

In [ ]:
ut.dir_performance(normalize='whole_process')

In [ ]:
print("total pages analysed in this sample (english):", len(SAMPLE_PAGES))
print(f"{len(SAMPLE_PAGES)/len(WPP_PAGES)*100:.2f}% of the total")

In [ ]:
print("total pages analysed in this sample (en+ot):", ut.TEMP_COUNTERS['pipeline_sources'])
# print(f"{ut.TEMP_COUNTERS['_sources_extraction']/EXTIMATED_TOT_REVISIONS*100:.2f}% of the total")

In [ ]:
print("total revisions analysed in this sample (en+ot):", ut.TEMP_COUNTERS['_sources_extraction'])

In [ ]:
print("total time of this sample process:", ut.TEMP_TIMERS['whole_process']/60**2, 'hrs')

In [ ]:
print("average time per page (english):", 
      ut.TEMP_TIMERS['___pipeline_english']/(ut.TEMP_COUNTERS['___pipeline_english'])/60," min per page")

In [ ]:
print("average time per page (other_languages):", 
      ut.TEMP_TIMERS['___pipeline_other_languages']/(ut.TEMP_COUNTERS['___pipeline_other_languages'])/60," min per page")

In [ ]:
print("average time per revision (english+other_languages):",\
      ut.TEMP_TIMERS['whole_process']/(ut.TEMP_COUNTERS['_sources_extraction'])," seconds per revision (including request time)")

In [ ]:
print("average time TO PROCESS one revision:", ut.TEMP_TIMERS['_sources_extraction']/(ut.TEMP_COUNTERS['_sources_extraction'])," seconds per revision")

# Extimate by revision (if tot revision in known)

#### Extimate total time
- expected_tot revisions * average revision time =expected_tot_revisions * (sample_n_revisions/sample_process_time)
- - sample_n_revisions is given by ut.TEMP_COUNTERS(\'_sources_extraction\') - tracing get_sources2locations calls
- - sample_process_time is ut.TEMP_TIMERS('whole_process') (accounting for request time as well)
- - expected_tot_revisions: counting them in wpp_to_n_revisions.ipynb

In [ ]:
# unknown for other languages
# EXTIMATED_TOT_REVISIONS=1483462 #manually added from results of wpp_to_n_revisions

<!-- Can be an overextimation, since the time to process one revision is dependent on page length and number of templates/ref statements to be found -->

In [ ]:
# print("extimated total time:", 
#       EXTIMATED_TOT_REVISIONS*ut.TEMP_TIMERS['whole_process']/(ut.TEMP_COUNTERS['_sources_extraction'])/60**2, 'hrs')

<!-- With the sample of the first 40 page, I expect less time since the first one have more edits and more sources inside -->

Info to retain

- **SECTION WHERE THE SOURCE IS USED?** Worthwile I guess
- **WHAT TO DO WITH PAGES THAT ARE JUST REDIRECT?** Not worthwile I guess
- flag some revision? can be done later 
- bot activity? can be done later

Not clear to me (wiki api)

- I do get 50 revision per request, while rvlim='max' is supposed to provide 500 (works with 500 for the wpp_to_n_revisions indeed)
- Is there a request limit for wikipedia action API?
Yes? https://api.wikimedia.org/wiki/Documentation/Getting_started/Rate_limits#Anonymous_requests 
- - Am I anonymus?
- - is this rate limit applicable to this request?

Better pipeline
- Parallelization 
- Asyncronous process to save request time (around 20% time decrease)
- sources2locs algorithm
- - retrieve page differences before comparing versions?
- - ignore somewhat bad revision? [generally no because that is a signal from a user] Maybe revisions flagged as "minor"? If we are sure that a source is not modified in minor revisions

structured data:

- urlsrv dataframe
- - identifier of page (is it implicit in revision_id?)
- - identifier of user (is it implicit in revision_id? Is it worth to de-structure it away?)
- - identifier of revision
- - identifier of url
- - added/removed 

- Page data:
- - identifier of page
- - page id, page name
- - whatever

- User data:
- - identifier of user
- - user_id, username
- - whatever

- Revision data
- - identifier of revision
- - parent_id
- - timestamp
- - user?
- - page?

- URLS data:
- - identifier of url
- - url status
- - domain
- - destination_url
- - destination_domain 

